In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
# import sys
# !{sys.executable} -m pip install tensorflow-gpu==2.0a0

In [3]:
import os
import pandas as pd
import networkx as nx

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import shuffle
import os

In [4]:
tf.__version__

'2.0.0-alpha0'

In [5]:
file_folder = '../../data/input'
os.listdir(file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [7]:
# train = pd.read_csv(f'{file_folder}/train.csv')
# magnetic_shielding_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
# mulliken_charges = pd.read_csv(f'{file_folder}/mulliken_charges.csv')
# potential_energy = pd.read_csv(f'{file_folder}/potential_energy.csv')
# scalar_coupling_contributions = pd.read_csv(f'{file_folder}/scalar_coupling_contributions.csv')
# dipole_moments = pd.read_csv(f'{file_folder}/dipole_moments.csv')
# structures = pd.read_csv(f'{file_folder}/structures.csv')
# test = pd.read_csv(f'{file_folder}/test.csv')

In [8]:
# from sklearn.preprocessing import LabelEncoder

# col = 'type'
# le = LabelEncoder()
# le.fit(list(train[col].values) + list(test[col].values))
# le.classes_
['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']

['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']

In [9]:
# for col in train.columns:
#     col_type = df_train[col].dtypes
#     if not col_type in numerics:
#         print(col, df_train[col].unique())
#         le = LabelEncoder()
#         le.fit(list(df_train[col].values) + list(df_test[col].values))
#         df_train[col] = le.transform(list(df_train[col].values))
#         df_test[col] = le.transform(list(df_test[col].values))
#         print(le.classes_)

In [10]:
datadir = "../../data/npz"

nodes_train     = np.load(datadir + "/nodes_train.npz" )['arr_0']
in_edges_train  = np.load(datadir + "/in_edges_train.npz")['arr_0']
out_edges_train = np.load(datadir + "/out_edges_train.npz" )['arr_0']

# nodes_test     = np.load(datadir + "/nodes_test.npz" )['arr_0']
# in_edges_test  = np.load(datadir + "/in_edges_test.npz")['arr_0']

In [ ]:
nodes_train.shape, in_edges_train.shape, out_edges_train.shape

In [11]:
out_labels = out_edges_train.reshape(-1,out_edges_train.shape[1]*out_edges_train.shape[2],1)
in_edges_train = in_edges_train.reshape(-1,in_edges_train.shape[1]*in_edges_train.shape[2],in_edges_train.shape[3])
# in_edges_test  = in_edges_test.reshape(-1,in_edges_test.shape[1]*in_edges_test.shape[2],in_edges_test.shape[3])

In [12]:
# nodes_train, in_edges_train, out_labels = shuffle(nodes_train, in_edges_train, out_labels)

MemoryError: 

In [13]:
class Message_Passer_NNM(tf.keras.layers.Layer):
    def __init__(self, node_dim):
        super(Message_Passer_NNM, self).__init__()
        self.node_dim = node_dim
        self.nn = tf.keras.layers.Dense(units=self.node_dim*self.node_dim, activation = tf.nn.relu)
      
    def call(self, node_j, edge_ij):
        
        # Embed the edge as a matrix
        A = self.nn(edge_ij)
        
        # Reshape so matrix mult can be done
        A = tf.reshape(A, [-1, self.node_dim, self.node_dim])
        node_j = tf.reshape(node_j, [-1, self.node_dim, 1])
        
        # Multiply edge matrix by node and shape into message list
        messages = tf.linalg.matmul(A, node_j)
        messages = tf.reshape(messages, [-1, tf.shape(edge_ij)[1], self.node_dim])

        return messages

In [14]:
class Message_Agg(tf.keras.layers.Layer):
    def __init__(self):
        super(Message_Agg, self).__init__()
    
    def call(self, messages):
        return tf.math.reduce_sum(messages, 2)

In [15]:
class Update_Func_GRU(tf.keras.layers.Layer):
    def __init__(self, state_dim):
        super(Update_Func_GRU, self).__init__()
        self.concat_layer = tf.keras.layers.Concatenate(axis=1)
        self.GRU = tf.keras.layers.GRU(state_dim)
        
    def call(self, old_state, agg_messages):
    
        # Remember node dim
        n_nodes  = tf.shape(old_state)[1]
        node_dim = tf.shape(old_state)[2]
        
        # Reshape so GRU can be applied, concat so old_state and messages are in sequence
        old_state = tf.reshape(old_state, [-1, 1, tf.shape(old_state)[-1]])
        agg_messages = tf.reshape(agg_messages, [-1, 1, tf.shape(agg_messages)[-1]])
        concat = self.concat_layer([old_state, agg_messages])
        
        # Apply GRU and then reshape so it can be returned
        activation = self.GRU(concat)
        activation = tf.reshape(activation, [-1, n_nodes, node_dim])
        
        return activation

In [16]:
class Edge_Regressor(tf.keras.layers.Layer):
    def __init__(self, intermediate_dim):
        super(Edge_Regressor, self).__init__()
        self.concat_layer = tf.keras.layers.Concatenate()
        self.hidden_layer_1 = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.hidden_layer_2 = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.output_layer = tf.keras.layers.Dense(units=1, activation=None)

        
    def call(self, nodes, edges):
            
        # Remember node dims
        n_nodes  = tf.shape(nodes)[1]
        node_dim = tf.shape(nodes)[2]
        
        # Tile and reshape to match edges
        state_i = tf.reshape(tf.tile(nodes, [1, 1, n_nodes]),[-1,n_nodes*n_nodes, node_dim ])
        state_j = tf.tile(nodes, [1, n_nodes, 1])
        
        # concat edges and nodes and apply MLP
        concat = self.concat_layer([state_i, edges, state_j])
        activation_1 = self.hidden_layer_1(concat)  
        activation_2 = self.hidden_layer_2(activation_1)

        return self.output_layer(activation_2)

In [17]:
class MP_Layer(tf.keras.layers.Layer):
    def __init__(self, state_dim):
        super(MP_Layer, self).__init__(self)
        self.message_passers  = Message_Passer_NNM(node_dim = state_dim) 
        self.message_aggs    = Message_Agg()
        self.update_functions = Update_Func_GRU(state_dim = state_dim)
        
        self.state_dim = state_dim         

    def call(self, nodes, edges, mask):
      
        n_nodes  = tf.shape(nodes)[1]
        node_dim = tf.shape(nodes)[2]
        
        state_j = tf.tile(nodes, [1, n_nodes, 1])

        messages  = self.message_passers(state_j, edges)

        # Do this to ignore messages from non-existant nodes
        masked =  tf.math.multiply(messages, mask)
        
        masked = tf.reshape(masked, [tf.shape(messages)[0], n_nodes, n_nodes, node_dim])

        agg_m = self.message_aggs(masked)
        
        updated_nodes = self.update_functions(nodes, agg_m)
        
        nodes_out = updated_nodes
        # Batch norm seems not to work. 
        #nodes_out = self.batch_norm(updated_nodes)
        
        return nodes_out

In [18]:
adj_input = tf.keras.Input(shape=(None,), name='adj_input')
nod_input = tf.keras.Input(shape=(None,), name='nod_input')
class MPNN(tf.keras.Model):
    def __init__(self, out_int_dim, state_dim, T):
        super(MPNN, self).__init__(self)   
        self.T = T
        self.embed = tf.keras.layers.Dense(units=state_dim, activation=tf.nn.relu)
        self.MP = MP_Layer( state_dim)     
        self.edge_regressor  = Edge_Regressor(out_int_dim)
        self.batch_norm = tf.keras.layers.BatchNormalization() 

        
    def call(self, inputs =  [adj_input, nod_input]):
      
      
        nodes            = inputs['nod_input']
        edges            = inputs['adj_input']

        # Get distances, and create mask wherever 0 (i.e. non-existant nodes)
        # This also masks node self-interactions...
        # This assumes distance is last
        len_edges = tf.shape(edges)[-1]
        
        _, x = tf.split(edges, [len_edges -1, 1], 2)
        mask =  tf.where(tf.equal(x, 0), x, tf.ones_like(x))
        
        # Embed node to be of the chosen node dimension (you can also just pad)
        nodes = self.embed(nodes) 
        
        nodes = self.batch_norm(nodes)
        # Run the T message passing steps
        for mp in range(self.T):
            nodes =  self.MP(nodes, edges, mask)
        
        # Regress the output values
        con_edges = self.edge_regressor(nodes, edges)
        
        
        return con_edges

In [19]:
def mse(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)


    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(nums, preds)))


    return reconstruction_error

In [20]:
def log_mse(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)


    reconstruction_error = tf.math.log(tf.reduce_mean(tf.square(tf.subtract(nums, preds))))


    return reconstruction_error

In [21]:
def mae(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)


    reconstruction_error = tf.reduce_mean(tf.abs(tf.subtract(nums, preds)))


    return reconstruction_error

In [22]:
def log_mae(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)

    reconstruction_error = tf.math.log(tf.reduce_mean(tf.abs(tf.subtract(nums, preds))))

    return reconstruction_error

In [23]:
learning_rate = 0.001
def step_decay(epoch):
    initial_lrate = learning_rate
    drop = 0.1
    epochs_drop = 20.0
    lrate = initial_lrate * np.power(drop,  
           np.floor((epoch)/epochs_drop))
    tf.print("Learning rate: ", lrate)
    return lrate

lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 15, restore_best_weights=True)

#lrate  =  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
#                              patience=5, min_lr=0.00001, verbose = 1)

opt = tf.optimizers.Adam(learning_rate=learning_rate)

In [24]:
mpnn = MPNN(out_int_dim = 256, state_dim = 64, T = 2)
mpnn.compile(opt, log_mae, metrics = [mae, log_mse])

In [25]:
train_size = int(len(out_labels)*0.8)
batch_size = 16
epochs = 25

In [26]:
mpnn.call({'adj_input' : in_edges_train[:10], 'nod_input': nodes_train[:10]})

<tf.Tensor: id=653, shape=(10, 841, 1), dtype=float32, numpy=
array([[[-0.25252724],
        [-0.46343338],
        [-0.47184452],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[-0.42281568],
        [-1.4393371 ],
        [-1.4949441 ],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[-0.00225324],
        [-0.14406522],
        [-0.14406522],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       ...,

       [[-0.1604631 ],
        [-0.26247677],
        [-0.2393753 ],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[-0.12093869],
        [-0.06156129],
        [-0.09681971],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[-0.23534523],
        [-0.28116727],
        [-0.3592159 ],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]]], dtype=f

In [27]:
train_size

68002

In [32]:
mpnn.fit({'adj_input' : in_edges_train[:train_size], 'nod_input': nodes_train[:train_size]}, y = out_labels[:train_size], batch_size = batch_size, epochs = 200, 
         callbacks = [lrate, stop_early], use_multiprocessing = True, initial_epoch = 0, verbose = 2, 
         validation_data = ({'adj_input' : in_edges_train[train_size:], 'nod_input': nodes_train[train_size:]},out_labels[train_size:]) )

Learning rate:  0.001
Epoch 1/200
68002/68002 - 424s - loss: -9.6600e-01 - mae: 0.3828 - log_mse: -1.0289e+00 - val_loss: -9.5943e-01 - val_mae: 0.4180 - val_log_mse: -1.1082e+00
Learning rate:  0.001
Epoch 2/200
68002/68002 - 424s - loss: -9.7458e-01 - mae: 0.3796 - log_mse: -1.0468e+00 - val_loss: -9.8226e-01 - val_mae: 0.4041 - val_log_mse: -1.1713e+00
Learning rate:  0.001
Epoch 3/200
68002/68002 - 421s - loss: -9.8585e-01 - mae: 0.3753 - log_mse: -1.0688e+00 - val_loss: -9.6591e-01 - val_mae: 0.4121 - val_log_mse: -1.1448e+00
Learning rate:  0.001
Epoch 4/200
68002/68002 - 421s - loss: -9.9159e-01 - mae: 0.3732 - log_mse: -1.0809e+00 - val_loss: -9.5818e-01 - val_mae: 0.4123 - val_log_mse: -1.1261e+00
Learning rate:  0.001
Epoch 5/200
68002/68002 - 423s - loss: -1.0006e+00 - mae: 0.3697 - log_mse: -1.1008e+00 - val_loss: -9.5868e-01 - val_mae: 0.4141 - val_log_mse: -1.1091e+00
Learning rate:  0.001
Epoch 6/200
68002/68002 - 420s - loss: -1.0086e+00 - mae: 0.3668 - log_mse: -1.1167

Learning rate:  1.0000000000000003e-05
Epoch 46/200
68002/68002 - 420s - loss: -1.3975e+00 - mae: 0.2489 - log_mse: -1.8330e+00 - val_loss: -1.2865e+00 - val_mae: 0.3060 - val_log_mse: -1.7279e+00
Learning rate:  1.0000000000000003e-05
Epoch 47/200
68002/68002 - 421s - loss: -1.3977e+00 - mae: 0.2489 - log_mse: -1.8325e+00 - val_loss: -1.2868e+00 - val_mae: 0.3060 - val_log_mse: -1.7276e+00
Learning rate:  1.0000000000000003e-05
Epoch 48/200
68002/68002 - 419s - loss: -1.3984e+00 - mae: 0.2488 - log_mse: -1.8350e+00 - val_loss: -1.2863e+00 - val_mae: 0.3062 - val_log_mse: -1.7263e+00
Learning rate:  1.0000000000000003e-05
Epoch 49/200
68002/68002 - 420s - loss: -1.3986e+00 - mae: 0.2487 - log_mse: -1.8339e+00 - val_loss: -1.2868e+00 - val_mae: 0.3060 - val_log_mse: -1.7273e+00
Learning rate:  1.0000000000000003e-05
Epoch 50/200
68002/68002 - 423s - loss: -1.3986e+00 - mae: 0.2486 - log_mse: -1.8333e+00 - val_loss: -1.2865e+00 - val_mae: 0.3061 - val_log_mse: -1.7274e+00
Learning rate: 

KeyboardInterrupt: 

In [33]:
del nodes_train
del in_edges_train
del out_edges_train

# nodes_test     = np.load(datadir + "/nodes_test.npz" )['arr_0']
# in_edges_test  = np.load(datadir + "/in_edges_test.npz")['arr_0']

In [36]:
del out_labels
# del in_edges_train

NameError: name 'in_edges_train' is not defined

In [37]:
import gc
gc.collect()

179

In [ ]:
json_string = model.to_json()
model = model_from_json(json_string)

In [ ]:
mpnn

In [38]:
nodes_test     = np.load(datadir + "/nodes_test.npz" )['arr_0']
in_edges_test  = np.load(datadir + "/in_edges_test.npz")['arr_0']
in_edges_test  = in_edges_test.reshape(-1,in_edges_test.shape[1]*in_edges_test.shape[2],in_edges_test.shape[3])

In [39]:
preds = mpnn.predict({'adj_input' : in_edges_test, 'nod_input': nodes_test})

In [40]:
preds.shape

(45772, 841, 1)

In [41]:
train = pd.read_csv(file_folder + "/train.csv")
test = pd.read_csv(file_folder + "/test.csv")

test_group = test.groupby('molecule_name')

# scale_min  = train['scalar_coupling_constant'].min()
# scale_max = train['scalar_coupling_constant'].max()
# scale_mid = (scale_max + scale_min)/2
# scale_norm = scale_max - scale_mid

In [42]:
def make_outs(test_group, preds):
    i = 0
    x = np.array([])
    for test_gp, preds in zip(test_group, preds):
        if (not i%1000):
            print(i)

        gp = test_gp[1]
        
        x = np.append(x, (preds[gp['atom_index_0'].values, gp['atom_index_1'].values] + preds[gp['atom_index_1'].values, gp['atom_index_0'].values])/2.0)
        
        i = i+1
    return x

In [43]:
max_size = 29
preds = preds.reshape((-1,max_size, max_size))

In [44]:
out_unscaled = make_outs(test_group, preds)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


In [45]:
test['scalar_coupling_constant'] = out_unscaled
test['scalar_coupling_constant'] = test['scalar_coupling_constant']
# test[['id','scalar_coupling_constant']].to_csv('submission.csv', index=False)

In [47]:
test.tail()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
2505537,7163684,dsgdb9nsd_133885,15,3,2JHC,0.490522
2505538,7163685,dsgdb9nsd_133885,15,4,2JHC,3.348294
2505539,7163686,dsgdb9nsd_133885,15,6,3JHC,1.352033
2505540,7163687,dsgdb9nsd_133885,15,7,2JHC,4.983674
2505541,7163688,dsgdb9nsd_133885,15,8,1JHC,121.098145


In [48]:
test.shape

(2505542, 6)

In [49]:
test[['id','scalar_coupling_constant']].to_csv('../../data/submission/submission_mpnn_keras_.03.csv', index=False)